In [ ]:
import heapq

class Node:
    def __init__(self, position, parent=None):
        self.position = position  # (x, y) coordinates
        self.parent = parent
        self.g = 0  # Cost from start to node
        self.h = 0  # Heuristic cost to goal
        self.f = 0  # Total cost

    def __lt__(self, other):
        return self.f < other.f

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar_pathfinding(grid):
    start = (0, 0)
    goal = (4, 4)

    open_list = []
    closed_list = set()

    start_node = Node(start)
    goal_node = Node(goal)

    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.position == goal:
            # Path found
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            path.reverse()
            return path

        closed_list.add(current_node.position)

        for direction in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            neighbor_pos = (current_node.position[0] + direction[0], current_node.position[1] + direction[1])

            if (0 <= neighbor_pos[0] < 5 and 0 <= neighbor_pos[1] < 5 and
                grid[neighbor_pos[0]][neighbor_pos[1]] == 0 and
                neighbor_pos not in closed_list):

                neighbor_node = Node(neighbor_pos, current_node)
                neighbor_node.g = current_node.g + 1
                neighbor_node.h = manhattan_distance(neighbor_pos, goal)
                neighbor_node.f = neighbor_node.g + neighbor_node.h

                if all(neighbor_node.f < n.f for n in open_list if n.position == neighbor_pos):
                    heapq.heappush(open_list, neighbor_node)

    return None  # No path found

def mark_path(grid, path):
    for pos in path:
        grid[pos[0]][pos[1]] = 'P'
    return grid

# Example grid (1 = obstacle, 0 = free space)
grid = [
    [0, 0, 0, 1, 0],
    [1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

path = astar_pathfinding(grid)
if path:
    marked_grid = mark_path(grid, path)
    for row in marked_grid:
        print(row)
else:
    print("No path found.")


['P', 'P', 0, 1, 0]
[1, 'P', 1, 1, 0]
[0, 'P', 'P', 'P', 'P']
[0, 1, 1, 1, 'P']
[0, 0, 0, 0, 'P']


In [ ]:
from collections import deque

class JugState:
    def __init__(self, jug1, jug2, parent=None):
        self.jug1 = jug1
        self.jug2 = jug2
        self.parent = parent
        self.path_cost = 0

def heuristic(state, target):
    return abs(state.jug1 - target) + abs(state.jug2 - target)

def a_star_water_jug(target):
    jug_capacity = (4, 3)
    start_state = JugState(0, 0)
    open_list = []
    closed_set = set()

    open_list.append((start_state, heuristic(start_state, target)))

    while open_list:
        current_state, current_cost = min(open_list, key=lambda x: x[1])
        open_list.remove((current_state, current_cost))

        if current_state.jug1 == target or current_state.jug2 == target:
            path = []
            while current_state:
                path.append((current_state.jug1, current_state.jug2))
                current_state = current_state.parent
            return path[::-1]

        closed_set.add((current_state.jug1, current_state.jug2))

        next_states = [
            JugState(jug_capacity[0], current_state.jug2, current_state),  # Fill jug1
            JugState(current_state.jug1, jug_capacity[1], current_state),  # Fill jug2
            JugState(0, current_state.jug2, current_state),  # Empty jug1
            JugState(current_state.jug1, 0, current_state),  # Empty jug2
            JugState(max(current_state.jug1 - (jug_capacity[1] - current_state.jug2), 0),
                        min(jug_capacity[1], current_state.jug1 + current_state.jug2), current_state),  # Pour jug1 into jug2
            JugState(min(current_state.jug1 + current_state.jug2, jug_capacity[0]),
                        max(current_state.jug2 - (jug_capacity[0] - current_state.jug1), 0), current_state)  # Pour jug2 into jug1
        ]

        for next_state in next_states:
            if (next_state.jug1, next_state.jug2) not in closed_set:
                cost = next_state.path_cost + heuristic(next_state, target)
                open_list.append((next_state, cost))

    return None  # No solution found

target_amount = 2
solution_path = a_star_water_jug(target_amount)

if solution_path:
    print("Steps to measure 2 liters:")
    for state in solution_path:
        print(f"Jug1: {state[0]}, Jug2: {state[1]}")
else:
    print("No solution found.")


Steps to measure 2 liters:
Jug1: 0, Jug2: 0
Jug1: 0, Jug2: 3
Jug1: 3, Jug2: 0
Jug1: 3, Jug2: 3
Jug1: 4, Jug2: 2


In [ ]:
import random

class Board:
    def __init__(self, queens):
        self.queens = queens  # List of column positions for each row
        self.conflicts = self.calculate_conflicts()

    def calculate_conflicts(self):
        """Calculate the number of conflicts (attacking pairs of queens)."""
        conflicts = 0
        for i in range(len(self.queens)):
            for j in range(i + 1, len(self.queens)):
                if self.queens[i] == self.queens[j] or abs(self.queens[i] - self.queens[j]) == abs(i - j):
                    conflicts += 1
        return conflicts

    def get_successors(self):
        """Generate all possible successors (moves) of the current board."""
        successors = []
        for row in range(len(self.queens)):
            for col in range(len(self.queens)):
                if col != self.queens[row]:  # Don't move to the current position
                    new_board = self.queens[:]
                    new_board[row] = col
                    successors.append(Board(new_board))
        return successors

    def __str__(self):
        """Return a string representation of the board."""
        size = len(self.queens)
        board_str = ""
        for i in range(size):
            board_str += "." * self.queens[i] + "Q" + "." * (size - self.queens[i] - 1) + "\n"
        return board_str.strip()

def hill_climbing():
    # Initial random state
    initial_queens = [random.randint(0, 7) for _ in range(8)]
    current_board = Board(initial_queens)

    while current_board.conflicts > 0:
        successors = current_board.get_successors()
        next_board = min(successors, key=lambda b: b.conflicts, default=None)

        if next_board.conflicts >= current_board.conflicts:
            # No better move found, return current board (local maximum)
            return current_board

        current_board = next_board

    return current_board  # Found a solution

# Run the hill climbing algorithm
solution_board = hill_climbing()

if solution_board.conflicts == 0:
    print("Solution found:")
    print(solution_board)
else:
    print("No solution found. Local maximum reached.")


Solution found:
......Q.
...Q....
.Q......
.......Q
.....Q..
Q.......
..Q.....
....Q...


In [2]:
import math

# Constants
PLAYER_X = "X"
PLAYER_O = "O"
EMPTY = " "

class TicTacToe:
    def __init__(self):
        self.board = [[EMPTY] * 3 for _ in range(3)]

    def print_board(self):
        for row in self.board:
            print("|".join(row))
            print("-" * 5)

    def is_winner(self, player):
        # Check rows, columns, and diagonals for a winner
        for i in range(3):
            if all(cell == player for cell in self.board[i]):  # Check rows
                return True
            if all(self.board[j][i] == player for j in range(3)):  # Check columns
                return True
        if all(self.board[i][i] == player for i in range(3)):  # Check diagonal
            return True
        if all(self.board[i][2 - i] == player for i in range(3)):  # Check anti-diagonal
            return True
        return False

    def is_full(self):
        return all(cell != EMPTY for row in self.board for cell in row)

    def mini_max(self, is_maximizing):
        if self.is_winner(PLAYER_O):
            return 1  # O wins
        if self.is_winner(PLAYER_X):
            return -1  # X wins
        if self.is_full():
            return 0  # Draw

        if is_maximizing:
            best_score = -math.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == EMPTY:
                        self.board[i][j] = PLAYER_O
                        score = self.mini_max(False)
                        self.board[i][j] = EMPTY
                        best_score = max(score, best_score)
            return best_score
        else:
            best_score = math.inf
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == EMPTY:
                        self.board[i][j] = PLAYER_X
                        score = self.mini_max(True)
                        self.board[i][j] = EMPTY
                        best_score = min(score, best_score)
            return best_score

    def best_move(self):
        best_score = -math.inf
        move = (-1, -1)
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == EMPTY:
                    self.board[i][j] = PLAYER_O
                    score = self.mini_max(False)
                    self.board[i][j] = EMPTY
                    if score > best_score:
                        best_score = score
                        move = (i, j)
        return move

    def play(self):
        while True:
            self.print_board()
            if self.is_winner(PLAYER_X):
                print("Player X wins!")
                break
            if self.is_full():
                print("It's a draw!")
                break

            # Player X move
            x, y = map(int, input("Enter your move (row and column): ").split())
            if self.board[x][y] != EMPTY:
                print("Invalid move. Try again.")
                continue
            self.board[x][y] = PLAYER_X

            # Check for win/draw
            if self.is_winner(PLAYER_X):
                self.print_board()
                print("Player X wins!")
                break
            if self.is_full():
                self.print_board()
                print("It's a draw!")
                break

            # Computer move
            print("Computer is making a move...")
            move = self.best_move()
            self.board[move[0]][move[1]] = PLAYER_O

            if self.is_winner(PLAYER_O):
                self.print_board()
                print("Computer O wins!")
                break
            if self.is_full():
                self.print_board()
                print("It's a draw!")
                break

# Start the game
if __name__ == "__main__":
    game = TicTacToe()
    game.play()



 | | 
-----
 | | 
-----
 | | 
-----
Enter your move (row and column): 1 1
Computer is making a move...
O| | 
-----
 |X| 
-----
 | | 
-----
Enter your move (row and column): 0 1
Computer is making a move...
O|X| 
-----
 |X| 
-----
 |O| 
-----
Enter your move (row and column): 0 2
Computer is making a move...
O|X|X
-----
 |X| 
-----
O|O| 
-----
Enter your move (row and column): 1 0
Computer is making a move...
O|X|X
-----
X|X| 
-----
O|O|O
-----
Computer O wins!
